Os arquivos de dados para os codigos abaixo nao foram versionados por serem muito grandes

In [1]:
import os
import sys

from pyspark.sql import SparkSession

# Required for Spark to find Python executable
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession \
    .builder \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .appName("App Estabelecimento CNAES") \
    .getOrCreate()

spark.version

KeyboardInterrupt: 

In [ ]:
spark.stop()

In [ ]:
df_estabelecimentos = (spark
                       .read
                       .csv('dados/estabelecimentos/estabelecimentos',
                            header=True,
                            sep=';',
                            inferSchema=True))

Quantas colunas existem e quantas sao identificadas como numero

30 e 0

In [ ]:
len(df_estabelecimentos.columns)

In [ ]:
df_estabelecimentos.printSchema()

Quantidade de estabelecimentos

In [ ]:
(df_estabelecimentos
 .select('CNPJ_BASICO')
 # .distinct()
 .count())

Qual e o tamanho do arquivo em Parquet

CSV: 3,93G <br>
Parquet: 1,54G <br>
Result: 2,5 de reducao

In [ ]:
(df_estabelecimentos
 .write
 .mode('overwrite')
 .parquet("estabelecimentos.parquet"))

Quantos estabelecimentos não tem logradouro cadastrado

In [ ]:
df_estabelecimentos.createOrReplaceTempView('estabelecimentos')

In [ ]:
spark.sql("SELECT COUNT(*) "
          "FROM estabelecimentos "
          "WHERE LOGRADOURO IS NULL").show()

In [ ]:
def is_avenida(logradouro: str):
    return logradouro.upper().startswith('AVENIDA')


spark.udf.register('is_avenida', is_avenida)


In [ ]:
spark.sql("SELECT COUNT(*) "
          "FROM estabelecimentos "
          "WHERE logradouro IS NOT NULL "
          "AND is_avenida(logradouro) == TRUE").show()

Quantos CEPs distintos existem entre os estabelecimentos?

In [ ]:
(spark
 .sql("SELECT COUNT(DISTINCT cep) "
      "FROM estabelecimentos")
 .show())

Quantos estabelecimentos são filiais? Consulte o dicionário dos dados e use Spark SQL.

In [ ]:
(spark
 .sql("SELECT COUNT(*) "
      "FROM estabelecimentos "
      "WHERE IDENTIFICADOR_MATRIZ_FILIAL = 2")
 .show())

Quantos CNAEs existem na tabela CNAEs?

In [ ]:
df_cnaes = (spark
            .read
            .csv('dados/cnaes/cnaes.csv',
                 header=True,
                 sep=';',
                 inferSchema=True))

In [ ]:
df_cnaes.count()

Quantos estabelecimentos possuem um CNAE relacionado a cultivo

In [ ]:
df_estabelecimentos_cnaes = (df_estabelecimentos
                             .join(df_cnaes,
                                   df_cnaes.CNAE == df_estabelecimentos.CNAE_PRINCIPAL,
                                   how='left'))

In [ ]:
df_estabelecimentos_cnaes.show(2)

In [ ]:
def is_cnae_cultivo(descricao_cnae: str):
    return descricao_cnae.upper().startswith('CULTIVO')


spark.udf.register('is_cnae_cultivo', is_cnae_cultivo)

In [ ]:
df_estabelecimentos_cnaes.createOrReplaceTempView('estabelecimentos_cnaes')

In [ ]:
spark.sql("SELECT COUNT(*) FROM estabelecimentos_cnaes "
          "WHERE is_cnae_cultivo(descricao_cnae) == TRUE").show()

Lista os CNAEs que possuem o texto CULTIVO em uma posicao que não seja o inicio do nome

In [ ]:
def is_cnae_cultivo_nao_inicio(descricao_cnae: str):
    return (not descricao_cnae.upper().startswith('CULTIVO')
            and 'CULTIVO' in descricao_cnae.upper())

spark.udf.register('is_cnae_cultivo_nao_inicio', is_cnae_cultivo_nao_inicio)

In [ ]:
df_cnaes.createOrReplaceTempView('cnaes')

In [ ]:
spark.sql("SELECT * FROM cnaes "
          "WHERE is_cnae_cultivo_nao_inicio(descricao_cnae) == TRUE").show(3)